In [1]:
from pydub import AudioSegment
import numpy as np
import soundfile as sf
from scipy.fft import fft, ifft, fftfreq

In [5]:
def preprocess(input_file):
    audio_segment = AudioSegment.from_file(f'{input_file}.mp3', format="mp3")
    sample_rate = audio_segment.frame_rate
    audio = np.array(audio_segment.get_array_of_samples())
    if audio_segment.channels > 1:
        audio = audio.reshape((-1, audio_segment.channels))
        audio = np.mean(audio, axis=1)
    return audio, sample_rate

def wave_dispersion(input_file, max_dispersion):
    audio, sample_rate = preprocess(input_file)
    n = len(audio)
    freqs = fftfreq(n, 1 / sample_rate)
    spectrum = fft(audio)
    delays = max_dispersion * np.abs(freqs / freqs.max())
    phase_shifts = np.exp(-1j * 2 * np.pi * freqs * delays)
    shifted_spectrum = spectrum * phase_shifts
    dispersed_audio = np.real(ifft(shifted_spectrum))
    dispersed_audio = dispersed_audio / np.max(np.abs(dispersed_audio))
    sf.write(f'{input_file}_dispersed.wav', dispersed_audio, sample_rate)


wave_dispersion("Gardel_Volver", max_dispersion=10)
